# Regularization with SciKit-Learn

Previously we created a new polynomial feature set and then applied our standard linear regression on it, but we can be smarter about model choice and utilize regularization.

Regularization attempts to minimize the RSS (residual sum of squares) *and* a penalty factor. This penalty factor will penalize models that have coefficients that are too large. Some methods of regularization will actually cause non useful features to have a coefficient of zero, in which case the model does not consider the feature.

Let's explore two methods of regularization, Ridge Regression and Lasso. We'll combine these with the polynomial feature set (it wouldn't be as effective to perform regularization of a model on such a small original feature set of the original X).

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data and Setup

In [ ]:
df = pd.read_csv("Advertising.csv")
X = df.drop('sales',axis=1)
y = df['sales']

In [ ]:
X.head()

### Polynomial Conversion

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [ ]:
poly_features = polynomial_converter.fit_transform(X)

In [ ]:
poly_features.shape

In [ ]:
polynomial_converter.get_feature_names_out()

### Train | Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    poly_features, y, test_size=0.3, random_state=101)

----
----

## Scaling the Data

While our particular data set has all the values in the same order of magnitude ($1000s of dollars spent), typically that won't be the case on a dataset, and since the mathematics behind regularized models will sum coefficients together, its important to standardize the features. Review the theory videos for more info, as well as a discussion on why we only **fit** to the training data, and **transform** on both sets separately.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# help(StandardScaler)

In [ ]:
scaler = StandardScaler() # mean is 0 and std is 1

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train.mean()

In [ ]:
X_train.std()

In [ ]:
X_test.mean()

In [ ]:
X_test.std()

## Ridge Regression

Make sure to view video lectures for full explanation of Ridge Regression and choosing an alpha.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_model = Ridge(alpha=10)

In [ ]:
ridge_model.fit(X_train,y_train)

In [ ]:
test_predictions = ridge_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

In [ ]:
RMSE

How did it perform on the training set? (This will be used later on for comparison)

In [ ]:
# Training Set Performance
train_predictions = ridge_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

### Choosing an alpha value with Cross-Validation

Review the theory video for full details.

In [ ]:
from sklearn.linear_model import RidgeCV

In [ ]:
# help(RidgeCV)

In [ ]:
# Choosing a scoring: https://scikit-learn.org/stable/modules/model_evaluation.html
# Negative RMSE so all metrics follow convention "Higher is better"

# See all options: sklearn.metrics.SCORERS.keys()
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),scoring='neg_mean_absolute_error')

In [ ]:
# The more alpha options you pass, the longer this will take.
# Fortunately our data set is still pretty small
ridge_cv_model.fit(X_train,y_train)

In [ ]:
ridge_cv_model.alpha_

In [ ]:
test_predictions = ridge_cv_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

In [ ]:
RMSE

In [ ]:
# Training Set Performance
# Training Set Performance
train_predictions = ridge_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

In [ ]:
ridge_cv_model.coef_

In [ ]:
# ridge_cv_model.feature_names_in_

In [ ]:
ridge_cv_model.intercept_


-----

## Lasso Regression

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [ ]:
lasso_cv_model.fit(X_train,y_train)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
test_predictions = lasso_cv_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

In [ ]:
RMSE

In [ ]:
# Training Set Performance
# Training Set Performance
train_predictions = lasso_cv_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

In [ ]:
lasso_cv_model.coef_

## Elastic Net

Elastic Net combines the penalties of ridge regression and lasso in an attempt to get the best of both worlds!

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7,.9, .95, .99, 1],tol=0.01)

In [ ]:
elastic_model.fit(X_train,y_train)

In [ ]:
elastic_model.l1_ratio_

In [ ]:
test_predictions = elastic_model.predict(X_test)

In [ ]:
MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

In [ ]:
MAE

In [ ]:
RMSE

In [ ]:
# Training Set Performance
# Training Set Performance
train_predictions = elastic_model.predict(X_train)
MAE = mean_absolute_error(y_train,train_predictions)
MAE

In [ ]:
elastic_model.coef_

# Normalization

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

# Loading Data
df = pd.read_csv("Advertising.csv")

# Preparing input (X) and output (y)
X = df.drop('sales',axis=1)
y = df['sales']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=101)

print(np.array(X_train).min())
print(np.array(X_train).max())

print(np.array(X_test).min())
print(np.array(X_test).max())

# Applying Normalizer
normalizer = Normalizer()
normalizer.fit(X_train)
X_train = normalizer.transform(X_train)
X_test = normalizer.transform(X_test)

print("-----------")
print(X_train.min())
print(X_train.max())

print(X_test.min())
print(X_test.max())


-----
---